
# IMPORTING NECCESSARY LIBRARIES 

Importing preprocess ,it is a another file where i defined and execute all the main functions 

In [ ]:
import preprocess
import classify
import numpy as np
from PIL import Image
import cv2



Processing complete! Images saved.


# DEFINING THE IMAGE PATH

In [ ]:
# Define the image file name
image_path = "input.jpeg" 


 # PREPROCESSING THE IMAGE 

In [ ]:
def preprocess_img(image_path):
    img = cv2.imread(image_path)  # Read image using OpenCV
    result_img, transitional_imgs, components, rects = preprocess.driver_preprocess(img)
    return result_img, transitional_imgs, components, rects


# CLASSIFYING THE COMPONENT

In [ ]:
def classify_components(list_comp, image_path, rects):
    """Loads an image and classifies detected components."""
    img = np.array(Image.open(image_path))  # Open image using PIL
    classified, annotated_img = classify.driver_classify(list_comp, img, rects)
    return classified, annotated_img

# MAIN FILE 

In [7]:
def main():
    # Step 1: Preprocess the image
    result_img, transitional_imgs, components, rects = preprocess_img(image_path)

    # Step 2: Classify the detected components
    #classified, annotated_img = classify_components(components, IMAGE_PATH, rects)

    # Step 3: Save and show results
    cv2.imwrite("processed_circuit.jpg", result_img)
   # cv2.imwrite("annotated_circuit.jpg", annotated_img)
    print("Processing complete! Images saved.")

if __name__ == '__main__':
    main()

Processing complete! Images saved.


NOW the image which is preprocessed gets saved 